# Import Library

In [1]:
import pandas as pd 
import tweepy 
import nltk
import re

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from wordcloud import WordCloud

Disini saya Import Library yang akan saya gunakan

In [2]:
consumer_key=''
consumer_secret=''

access_token=''
access_token_secret=''

auth=tweepy.OAuthHandler(consumer_key,consumer_secret) #Akses akun twitter
auth.set_access_token(access_token,access_token_secret) #Akses twitter general
api = tweepy.API(auth,wait_on_rate_limit=True) #Autentikasi twitter API

Akses Twitter API Token

In [2]:
def scrappe(keyword,count,since,until): #Input: kata kunci, jumlah tweet yang diinginkan, tanggal awal, tanggal akhir
    text_query=f"since:{since} until:{until} {keyword}" #Query search twitter
    tweets = tweepy.Cursor(api.search,q=text_query).items(count) #Mencari tweet dengan kata kunci tertentu max seminggu sejak hari ini
    tweets_list = [[tweet.created_at, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, tweet.place] for tweet in tweets] #Mengambil parameter yang diperlukan
    df_tweet=pd.DataFrame(tweets_list)
    df_tweet.columns=['tanggal','id','text','rt','fav','place'] #tanggal: Tanggal tweet dipost, id: id tweet, text: isi tweet, rt: Jumlah retweet, fav: Jumlah favorit/like, coor: Koordinat lokasi
    return(df_tweet)

def proc_feature(features):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', features)
    processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    processed_feature = re.sub(r'^b\s+', '', processed_feature)
    processed_feature = processed_feature.lower()  
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    processed_feature = regrex_pattern.sub(r'',processed_feature)
    processed_feature = re.sub('rt',' ',processed_feature)
    processed_feature = re.sub('gak',' ',processed_feature)
    processed_feature = re.sub('http',' ',processed_feature)
    processed_feature = re.sub('https',' ',processed_feature)
    return(processed_feature)

Membuat Fungsi untuk scraping tweet dan fungsi text preprocessing

In [ ]:
data=scrappe("Data Indihome Bocor",10000,"2022-08-20","2022-08-24")
data.to_excel('Indihome.xlsx',index=False)

Masukan keyword yang akan dipilih

In [3]:
data=pd.read_excel('Indihome.xlsx')
data.head()

,tanggal,id,text,rt,fav,place
0,2022-08-23 23:42:50,1562223870467538944,@hyang_wisnu @kemkominfo @BSSN_RI @PlateJohnny...,0,1,NaN
1,2022-08-23 23:41:59,1562223659535986944,"RT @hyang_wisnu: DATA PLN DAN INDIHOME BOCOR, ...",16,0,NaN
2,2022-08-23 23:41:31,1562223542468775936,"Terkait kebocoran data PLN dan IndiHome bocor,...",0,0,NaN
3,2022-08-23 22:58:46,1562212781600177920,"RT @MMC_IDN2: 26 Juta Data Pribadi Diretas, Ne...",57,0,NaN
4,2022-08-23 22:57:33,1562212477093289984,RT @republikaonline: Data pribadi pelanggan PT...,2,0,NaN


Loading data yang telah di scraping untuk dilanjutkan ke proses sentiment skor

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4821 entries, 0 to 4820
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   tanggal  4821 non-null   datetime64[ns]
 1   id       4821 non-null   int64         
 2   text     4821 non-null   object        
 3   rt       4821 non-null   int64         
 4   fav      4821 non-null   int64         
 5   place    8 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 226.1+ KB


Info data yang telah discraping

In [5]:
print(data.text[0])

@hyang_wisnu @kemkominfo @BSSN_RI @PlateJohnny @jokowi @_pln_id @IndiHome @Aryprasetyo85 @Cahaya_Dfa @zhymme… https://t.co/3SJb0qTEwZ


In [5]:
#Load data list kata bersentimen negatif
neg_file=pd.read_csv('https://github.com/riochr17/Analisis-Sentimen-ID/raw/master/data/negatif.txt',header=None,delimiter='\t').values
neg_file=list(neg_file.reshape(neg_file.shape[0],))

#Load data list kata bersentimen positif
pst_file=pd.read_csv('https://github.com/riochr17/Analisis-Sentimen-ID/raw/master/data/positif.txt',header=None,delimiter='\t').values
pst_file=list(pst_file.reshape(pst_file.shape[0],))

Load data yang berisikan kata positif dan negatif

In [7]:
#Load data list stopwords bahasa indonesia
stp_word=pd.read_csv('https://github.com/stopwords-iso/stopwords-id/raw/master/stopwords-id.txt',header=None).values

Load data stopword

In [7]:
data['text']=data['text'].apply(lambda x: proc_feature(x)) #Filter karakter dan kata pada masing-masing data

In [9]:
#Pada bagian ini, dihitung jumlah kata negatif dan positif masing-masing tweet berdasarkan list kata pada neg_file dan pst_file

post_words_count=data['text'].apply(lambda x: len([w for w in x.split(' ') if w in pst_file]))
neg_words_count=data['text'].apply(lambda x: len([w for w in x.split(' ') if w in neg_file]))

Menghitung kata negatif dan positif

In [10]:
#Menghitung skor sentimen, selisih jml kata positif dan negatif
data['sent_score']=(post_words_count-neg_words_count)
#Memberi sign utk selisih <0 -1, >0 1, dan =0 0
data['sentiment']=data['sent_score'].apply(lambda x: 1 if x>0 else -1 if x<0 else 0)

Menghitung selisih jumlah kata positif dan negatif untuk mendapatkan sentiment skor

In [11]:
data

,tanggal,id,text,rt,fav,place,sent_score,sentiment
0,2022-08-23 23:42:50,1562223870467538944,hyang_wisnu kemkominfo bssn_ri platejohnny jo...,0,1,NaN,0,0
1,2022-08-23 23:41:59,1562223659535986944,hyang_wisnu data pln dan indihome bocor ini y...,16,0,NaN,-1,-1
2,2022-08-23 23:41:31,1562223542468775936,terkait kebocoran data pln dan indihome bocor ...,0,0,NaN,-2,-1
3,2022-08-23 22:58:46,1562212781600177920,mmc_idn2 26 juta data pribadi diretas negara ...,57,0,NaN,-1,-1
4,2022-08-23 22:57:33,1562212477093289984,republikaonline data pribadi pelanggan pt pln...,2,0,NaN,-2,-1
...,...,...,...,...,...,...,...,...
4816,2022-08-21 02:43:37,1561182202943382016,myname_idontk sepe biasa data privasi berbaga...,1,0,NaN,-1,-1
4817,2022-08-21 02:35:17,1561180106709577984,tilehopper itsdioandn isp logging data browsi...,7,0,NaN,0,0
4818,2022-08-21 02:33:21,1561179619902258944,tilehopper itsdioandn isp logging data browsi...,7,0,NaN,0,0
4819,2022-08-21 02:16:12,1561175305338768896,itsdioandn isp logging data browsing customer...,7,20,NaN,0,0


In [12]:
data.sentiment.value_counts()

-1    4243
 0     528
 1      50
Name: sentiment, dtype: int64

In [15]:
data.to_csv('Sentiment_Data_Indihome.csv',index=False)

Save data yang sudah dilakukan text proprocessing yang selanjutnya akan dilakukan modeling Deep Learning